In [15]:
%matplotlib inline
import pickle
import pandas as pd
from IPython.display import display

# Loading 2017 train data

In [16]:
with open('../data/processed/data.pkl', 'rb') as pf:
    objs = pickle.load(pf)

grammar_dic = objs[0]
df_17_train = objs[1]   # using RecResult
df_17_test = objs[2]    # using RecResult

recode = lambda x: 1 if x=='correct' else 0
df_17_train['language'] = df_17_train['language'].apply(recode)
df_17_test['language'] = df_17_test['language'].apply(recode)
df_17_train['meaning'] = df_17_train['meaning'].apply(recode)
df_17_test['meaning'] = df_17_test['meaning'].apply(recode)

# load various types of features

# df_17_train_vecsim = pd.read_csv('../data/processed/df17_train_fasttext.csv')
df_17_train_grmerr = pd.read_csv('../data/processed/df17_train_grmerror.csv')
# df_17_test_vecsim = pd.read_csv('../data/processed/df17_test_fasttext.csv')
df_17_test_grmerr = pd.read_csv('../data/processed/df17_test_grmerror.csv')

print(df_17_train.head(5))
print(df_17_test.shape)
# print(df_17_test_vecsim.shape)
print(df_17_test_grmerr.shape)

     Id                                  Prompt   Wavfile  \
0  5835               Frag: Zimmer für 4 Nächte  5835.wav   
1  5836                      Frag: Junior Suite  5836.wav   
2  5837                      Frag: Junior Suite  5837.wav   
3  5839  Sag: Ich möchte mit Postkarte bezahlen  5839.wav   
4  5840           Frag: Gibt es einen Coiffeur?  5840.wav   

                          RecResult                     Transcription  \
0            a room for four nights            a room for four nights   
1                      junior suite                    junior*z suite   
2                            junior                    kinderzimmer*v   
3  i would like to pay by post card  i would like to pay by post card   
4            is there a hairdresser            is there a hairdresser   

   language  meaning  
0         1        1  
1         1        1  
2         0        0  
3         1        1  
4         1        1  
(995, 7)
(995, 12)


## Form a DF for modeling
- df_17_X, only using Id, language, and meaning cols
- using Id to merge other features DFs


- 1/16/2018 Add more features provided by Huy
- 1/24/2018 Just keep one DF for langauge prediction and [meaning, language] labels 

In [17]:
df_17_train_huy = pd.read_csv('../data/processed/textProcessing_trainingKaldi_features.csv', sep='\t')
df_17_test_huy = pd.read_csv('../data/processed/textProcessing_testKaldi_annotated_features.csv',
                          sep = '\t')
df_17_train_huy = df_17_train_huy.drop(['CLASS'], axis=1)
df_17_test_huy = df_17_test_huy.drop(['CLASS'], axis=1)
print(df_17_train_huy.shape)
print(df_17_train_huy.columns)
print(df_17_test_huy.shape)

(5222, 50)
Index(['Id', 'ppl-ref', 'ppl-ref_pos', 'ppl-ref_prod', 'ppl-ref_dep',
       'ppl-prompt', 'ppl-prompt_pos', 'ppl-correct', 'ppl-correct_pos',
       'ppl-correct_prod', 'ppl-correct_dep', 'ppl-ge', 'ppl-ge_pos',
       'ppl-incorrect', 'ppl-incorrect_pos', 'ppl-incorrect_prod',
       'ppl-incorrect_dep', 'maxsim_15_skip', 'maxsim_30_skip',
       'maxsim_50_skip', 'maxsim_15_cbw', 'maxsim_30_cbw', 'maxsim_50_cbw',
       'lda_sim-max', 'lda_sim-min', 'lda_sim-avg', 'ngram_match',
       'ngram_match-lem', 'ngram_unseen-1', 'ngram_unseen-2', 'ngram_unseen-3',
       'error_yn', 'error_count', 'error_ratio', 'parse_score-ratio',
       'length_ratio', 'length_01', 'length_under-min', 'length_above-max',
       'length_unknown', 'length_unknown-ratio', 'length_sounds',
       'length_sounds-ratio', 'prompt_missing', 'prompt_missing-pct',
       'prompt_DT', 'prompt_IN', 'prompt_MD', 'prompt_NN', 'prompt_VB'],
      dtype='object')
(995, 50)


In [18]:
def gen_ml_df(df_main, df_grmerr, df_huy):
    df_ml = df_main[['Id', 'language', 'meaning']]
    # df_ml = pd.merge(df_ml, df_grmerr, on='Id')
    df_ml = pd.merge(df_ml, df_huy, on='Id')
    return df_ml

## Process 2017 train DF

In [19]:
df_17_train_ml = gen_ml_df(df_17_train, df_17_train_grmerr, df_17_train_huy)
display(df_17_train_ml.describe())
print(df_17_train_ml.shape)


,Id,language,meaning,ppl-ref,ppl-ref_pos,ppl-ref_prod,ppl-ref_dep,ppl-prompt,ppl-prompt_pos,ppl-correct,...,length_unknown-ratio,length_sounds,length_sounds-ratio,prompt_missing,prompt_missing-pct,prompt_DT,prompt_IN,prompt_MD,prompt_NN,prompt_VB
count,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,...,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000,5221.000000
mean,8797.797931,0.742961,0.896572,-28.470890,-6.312515,-13.310218,-18.897874,-4.043127,-4.160696,-19.035401,...,0.041400,5.513886,0.793526,1.473473,0.311583,0.154951,0.149397,0.088680,0.316031,0.279257
std,1695.111124,0.437043,0.304547,6.698166,3.372112,4.983533,6.753946,3.389601,3.309348,4.673358,...,0.122776,1.871967,0.404813,1.576745,0.324866,0.361893,0.356513,0.284309,0.464970,0.448677
min,5835.000000,0.000000,0.000000,-86.414010,-39.532380,-47.332080,-70.846450,-39.016860,-38.708110,-68.433020,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7344.000000,0.000000,1.000000,-30.966260,-6.566340,-16.364540,-22.269650,-3.809840,-3.932233,-20.251080,...,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8832.000000,1.000000,1.000000,-26.214450,-5.486407,-12.811660,-18.647690,-2.857069,-2.885361,-17.408310,...,0.000000,5.000000,1.000000,1.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10262.000000,1.000000,1.000000,-24.058410,-4.201643,-9.623078,-14.523600,-2.283301,-2.334454,-16.300390,...,0.000000,7.000000,1.000000,2.000000,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000
max,11676.000000,1.000000,1.000000,-22.034600,-0.924857,-3.289850,-1.974081,-0.602060,-0.602060,-14.283510,...,1.000000,20.000000,1.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(5221, 52)


## Process 2017 test DF

In [20]:
df_17_test_ml = gen_ml_df(df_17_test, df_17_test_grmerr, df_17_test_huy)
display(df_17_test_ml.describe())
print(df_17_test_ml.shape)

,Id,language,meaning,ppl-ref,ppl-ref_pos,ppl-ref_prod,ppl-ref_dep,ppl-prompt,ppl-prompt_pos,ppl-correct,...,length_unknown-ratio,length_sounds,length_sounds-ratio,prompt_missing,prompt_missing-pct,prompt_DT,prompt_IN,prompt_MD,prompt_NN,prompt_VB
count,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,...,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000
mean,4149.836181,0.719598,0.879397,-29.179244,-6.759619,-13.452600,-19.295123,-4.207140,-4.412498,-19.847525,...,0.053771,5.571859,0.718593,1.633166,0.348945,0.167839,0.119598,0.090452,0.368844,0.307538
std,325.004628,0.449422,0.325829,7.122178,4.169517,5.646571,7.684801,3.616838,3.539654,4.842951,...,0.123538,2.082700,0.449912,1.652723,0.334286,0.373911,0.324654,0.286973,0.482734,0.461706
min,3571.000000,0.000000,0.000000,-104.023400,-63.013410,-65.304610,-90.378650,-23.035050,-21.502150,-68.433020,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3869.500000,0.000000,1.000000,-31.674110,-7.734252,-16.670600,-22.686610,-3.910222,-4.364362,-21.462230,...,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4152.000000,1.000000,1.000000,-26.920760,-5.545169,-12.556460,-18.580800,-2.939943,-3.000676,-18.303180,...,0.000000,5.000000,1.000000,1.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4435.500000,1.000000,1.000000,-24.300880,-4.223809,-9.576643,-13.865990,-2.158363,-2.334454,-16.729550,...,0.000000,7.000000,1.000000,3.000000,0.571429,0.000000,0.000000,0.000000,1.000000,1.000000
max,4701.000000,1.000000,1.000000,-22.034600,-0.924857,-3.477604,-1.736662,-0.602060,-0.778151,-14.616090,...,1.000000,27.000000,1.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(995, 52)


## Correlation analysis
Only show |R| more than $CORR_{CUT}$

In [21]:
tb_corr = df_17_train_ml.corr()
CORR_CUT = 0.1

tb_l = tb_corr['language'].drop(['language', 'meaning'])
display(tb_l[tb_l > CORR_CUT])
display(tb_l[tb_l < -1 * CORR_CUT])
col_l = list(tb_l[tb_l > CORR_CUT].index) + list(tb_l[tb_l < -1 * CORR_CUT].index)
display(col_l)
len(col_l)

ppl-ref                0.412027
ppl-ref_pos            0.252779
ppl-ref_prod           0.150436
ppl-ref_dep            0.254020
ppl-prompt             0.161701
ppl-prompt_pos         0.158218
ppl-correct            0.448417
ppl-correct_pos        0.335484
ppl-correct_prod       0.202784
ppl-correct_dep        0.109184
ppl-ge                 0.102196
ppl-ge_pos             0.115325
ppl-incorrect_prod     0.110023
ppl-incorrect_dep      0.109184
maxsim_15_skip         0.295474
maxsim_30_skip         0.304698
maxsim_50_skip         0.301699
maxsim_15_cbw          0.299532
maxsim_30_cbw          0.309556
maxsim_50_cbw          0.307635
lda_sim-max            0.241508
lda_sim-min            0.111859
lda_sim-avg            0.180788
ngram_match            0.175150
ngram_match-lem        0.165776
length_sounds-ratio    0.104937
Name: language, dtype: float64

ppl-incorrect        -0.135479
ngram_unseen-1       -0.368065
ngram_unseen-2       -0.417670
length_under-min     -0.127047
length_above-max     -0.125318
length_sounds        -0.140345
prompt_missing       -0.370462
prompt_missing-pct   -0.280889
prompt_IN            -0.269888
prompt_MD            -0.134119
prompt_NN            -0.266681
prompt_VB            -0.223953
Name: language, dtype: float64

['ppl-ref',
 'ppl-ref_pos',
 'ppl-ref_prod',
 'ppl-ref_dep',
 'ppl-prompt',
 'ppl-prompt_pos',
 'ppl-correct',
 'ppl-correct_pos',
 'ppl-correct_prod',
 'ppl-correct_dep',
 'ppl-ge',
 'ppl-ge_pos',
 'ppl-incorrect_prod',
 'ppl-incorrect_dep',
 'maxsim_15_skip',
 'maxsim_30_skip',
 'maxsim_50_skip',
 'maxsim_15_cbw',
 'maxsim_30_cbw',
 'maxsim_50_cbw',
 'lda_sim-max',
 'lda_sim-min',
 'lda_sim-avg',
 'ngram_match',
 'ngram_match-lem',
 'length_sounds-ratio',
 'ppl-incorrect',
 'ngram_unseen-1',
 'ngram_unseen-2',
 'length_under-min',
 'length_above-max',
 'length_sounds',
 'prompt_missing',
 'prompt_missing-pct',
 'prompt_IN',
 'prompt_MD',
 'prompt_NN',
 'prompt_VB']

38

In [22]:
# convert pandas DF to numpy array

def get_langauge_X(df):
    X = df.loc[:, col_l].values
    return X

def get_langauge_y(df):
    return df['language'].values

def get_meaning_y(df):
    return df['meaning'].values

## z-norm all features

In [23]:
from sklearn.preprocessing.data import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1)) # for SVM
#scaler = StandardScaler()

In [24]:
year17_lang_train_X = get_langauge_X(df_17_train_ml)
year17_lang_train_y = get_langauge_y(df_17_train_ml)
year17_lang_test_X = get_langauge_X(df_17_test_ml)
year17_lang_test_y = get_langauge_y(df_17_test_ml)

scaler.fit(year17_lang_train_X)
year17_lang_train_X = scaler.transform(year17_lang_train_X)
year17_lang_test_X = scaler.transform(year17_lang_test_X)

year17_meaning_train_y = get_meaning_y(df_17_train_ml)
year17_meaning_test_y = get_meaning_y(df_17_test_ml)

print(year17_lang_train_X.shape)
print(year17_lang_train_y.shape)
print(year17_lang_test_X.shape)
print(year17_lang_test_y.shape)

(5221, 38)
(5221,)
(995, 38)
(995,)


Pickle all year17 numpy arraies

In [25]:
with open('../data/processed/numpy/year17_withHuy.pkl', 'wb') as pf:
    pickle.dump([year17_lang_train_X,
                 year17_lang_train_y,
                 year17_lang_test_X,
                 year17_lang_test_y,
                 year17_meaning_train_y,
                 year17_meaning_test_y], pf)
